# Onboarding Multiple CSVs

**Prerequisite**:

- Installed Neat, see [Installation](../../gettingstarted/installation.html)
- Launched a notebook environment.
- Familiar with the `NeatSession` object, see [introduction](../introduction/introduction.html)
- Access to `NeatEngine`.

In this tutorial, we will load data from two `csv` files, connect the data, infer a data model from the data and push the model with data to CDF.

## Reading Metadata

We will start by instansiating a `NeatSession` and read the data from multiple URLs.

In [1]:
from cognite.neat import NeatSession, get_cognite_client

In [2]:
# Note that we use Oxigraph in this example, this will not work in a CDF notebook
neat = NeatSession(get_cognite_client(".env"), storage="oxigraph")

Found .env file in repository root. Loaded variables from .env file.
Neat Engine 2.0.1 loaded.


In [3]:
base_url = "https://apps-cdn.cogniteapp.com/toolkit/publicdata/"

asset = "assets.Table.csv"
activity = "workitem.Table.csv"

neat.read.csv(f"{base_url}{asset}", type="Asset", primary_key="WMT_TAG_GLOBALID")
neat.read.csv(f"{base_url}{activity}", type="Activity", primary_key="sourceId")

In [4]:
neat

,Type,Occurrence
0,Activity,13002
1,Asset,1103


Studying the output above, we see that we succesfully read the assets and activities.

## Connecting Data

By studying the source data, we notice that the `WORKORDER_ITEMNAME` column in the activity CSV is referencing the `WMT_TAG_NAME` in the asset CSV.

We use neat to connect the activities to assets, through a new property we call `asset` in the activity.

In [5]:
source = ("Activity", "WORKORDER_ITEMNAME")
target = ("Asset", "WMT_TAG_NAME")
connection = "asset"

In [6]:
neat.prepare.instances.make_connection_on_exact_match(source, target, connection)

Found 100 connections. Adding them to the graph...


## Infer Data Model

We can infer a data model from data in the `NeatSession` by calling `.infer()`.

In [7]:
neat.infer()

,count
NeatIssue,
PropertySkippedWarning,1


In [8]:
neat

This gives us an unverified data model, which we can then verify.

## Verify Data Model

In [9]:
neat.verify()

You can inspect the issues with the .inspect.issues(...) method.


,count
NeatIssue,
PropertySkippedWarning,1
ResourceRegexViolationWarning,1


In [10]:
neat.inspect.issues()

### 2 issues found

  * **PropertySkippedWarning**: The Property with identifier Activity:externalId has a property externalId
    which is skipped. External ID is assumed to be the unique identifier of the instance and is not part of the data model schema..
  * **ResourceRegexViolationWarning**: The Property with identifier source.1 in the Properties sheet, Property column is violating
    the CDF regex (?!^(property|space|externalId|createdTime|lastUpdatedTime|deletedTime|edge_id|node_id|project_id|property_group|seq|tg_table_name|extensions)$)(^[a-zA-Z][a-zA-Z0-9_]{0,253}[a-zA-Z0-9]?$). This will lead to errors when converting to DMS data model.
    
Fix: Either export the data model and make the necessary changes manually or run prepare.cdf_compliant_external_ids.

We notice that one of the properies in the `Activity` contains a `.` which is illegal for CDF data models. Looking at the type hint we see that we can usea  prepare method to fix this issue.

In [11]:
neat.prepare.data_model.cdf_compliant_external_ids()

In [13]:
neat.verify()

[]

In [14]:
neat.convert("dms")

Rules converted to dms


[]

## Inspect Data Model

We use the `.show` to inspect the data model and see that the activity is connected to the asset

In [15]:
neat.set.data_model_id(("sp_doctrino2", "DoctrinoAssetActivityModel", "v1"))

Data model ID set to ('sp_doctrino2', 'DoctrinoAssetActivityModel', 'v1')


In [ ]:
neat.show.data_model()

<img src="../../artifacts/figs/connecting_metadata_dm.png" width="600">

## Publishing Data Model to CDF

Now we are ready to publish this to CDF.

In [17]:
neat.to.cdf.data_model()

You can inspect the details with the .inspect.outcome.data_model(...) method.


,name,created
0,schema,0
1,spaces,1
2,containers,2
3,views,2
4,data_models,1
5,nodes,0


<img src="../../artifacts/figs/connecting_metadata_published_dm.png" width="600">

## Populating Data Model

Neat keeps track of the data, so we can immidiately populate this data model with the original data

In [18]:
neat.to.cdf.instances()

INFO | 2024-12-06 15:37:01,060 | Staring DMSLoader and will process 2 views.
INFO | 2024-12-06 15:37:01,063 | Starting ViewId(space='sp_doctrino2', external_id='Activity', version='v1') 1/2.
INFO | 2024-12-06 15:37:44,886 | Finished ViewId(space='sp_doctrino2', external_id='Activity', version='v1').
INFO | 2024-12-06 15:37:44,888 | Starting ViewId(space='sp_doctrino2', external_id='Asset', version='v1') 2/2.
INFO | 2024-12-06 15:37:48,052 | Finished ViewId(space='sp_doctrino2', external_id='Asset', version='v1').


You can inspect the details with the .inspect.outcome.instances(...) method.


,name,created,changed
0,Activity,"13,002",0
1,Asset,"1,025",78


<img src="../../artifacts/figs/connecting_metadata_populated.png" width="1200">